# target만 clip한 데이터

In [1]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium
import time
from itertools import product 
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
import os
os.chdir('/Users/sujiyang/Desktop/aiservice/kaggle_future_sales')

In [3]:
data = pd.read_pickle('./data_1.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8032887 entries, 827691 to 8860577
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   month_id                    int8   
 1   shop_id                     int8   
 2   item_id                     int16  
 3   cnt_month                   float32
 4   city_id                     int8   
 5   cat_id                      int8   
 6   main_cat                    int8   
 7   sub_cat                     int8   
 8   year                        int16  
 9   month                       int8   
 10  sales_month                 int8   
 11  cnt_month_1                 float32
 12  cnt_month_2                 float32
 13  cnt_month_3                 float32
 14  item_id_mean_cnt_1          float32
 15  item_id_mean_cnt_2          float32
 16  item_id_mean_cnt_3          float32
 17  sub_cat_mean_cnt_1          float32
 18  sub_cat_mean_cnt_2          float32
 19  sub_cat_mean_cnt

In [4]:
data.isnull().sum()

month_id                           0
shop_id                            0
item_id                            0
cnt_month                     214200
city_id                            0
cat_id                             0
main_cat                           0
sub_cat                            0
year                               0
month                              0
sales_month                        0
cnt_month_1                        0
cnt_month_2                        0
cnt_month_3                        0
item_id_mean_cnt_1                 0
item_id_mean_cnt_2                 0
item_id_mean_cnt_3                 0
sub_cat_mean_cnt_1                 0
sub_cat_mean_cnt_2                 0
sub_cat_mean_cnt_3                 0
price_mean_mean_cnt_1              0
price_mean_mean_cnt_2              0
price_mean_mean_cnt_3              0
item_id/city_id_mean_cnt_1         0
item_id/city_id_mean_cnt_2         0
item_id/city_id_mean_cnt_3         0
shop_id/cat_id_mean_cnt_1          0
s

In [6]:
data.columns

Index(['month_id', 'shop_id', 'item_id', 'cnt_month', 'city_id', 'cat_id',
       'main_cat', 'sub_cat', 'year', 'month', 'sales_month', 'cnt_month_1',
       'cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1',
       'item_id_mean_cnt_2', 'item_id_mean_cnt_3', 'sub_cat_mean_cnt_1',
       'sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3',
       'item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1',
       'shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3'],
      dtype='object')

In [5]:
# 주어진 피쳐들의 조합으로 생성된 기준으로 평균 판매량 피쳐를 생성하고, cnt_month와의 상관계수를 보여주는 함수

def all_corr(data,features,num_features): # 데이터프레임, 사용할피쳐, 조합을 만들 피쳐 갯수
    # 결과값을 담을 리스트 생성
    result = []
    # 주어진 피쳐와 개수로 가능한 모든 조합 생성
    combi = list(itertools.combinations(features,num_features))
    
    for i in range(len(combi)):
        # 새로운 컬럼명 생성
        feature_name = '/'.join(combi[i])+'_mean_cnt'
        # 해당 조합의 가장 앞에 월 구분 id 추가  
        temp = list(combi[i])
        temp.insert(0,'month_id')
        # 해당 기준으로 월간 평균 판매량 컬럼 생성  
        df = mean_cnt_month(data,temp)
        # 생성된 데이터프레임내에서 상관계수 계산
        cor = df[[feature_name,'cnt_month']].corr().iloc[0,1]
        # 상관계수만 결과 리스트에 저장
        result.append([feature_name,cor])
        
    del combi, feature_name, temp, df, cor  
    return result

In [8]:
all_corr(data, month_id, shop_id, item_id, cnt_month, city_id, cat_id,
       main_cat, sub_cat, year, month, sales_month, cnt_month,2)

NameError: name 'month_id' is not defined

In [5]:
# train/validation/test data split
X_train = data[data['month_id'] < 33].drop(['cnt_month'], axis=1)
y_train = data[data['month_id'] < 33]['cnt_month'].clip(0,20)

X_valid = data[data['month_id'] == 33].drop(['cnt_month'], axis=1)
y_valid = data[data['month_id'] == 33]['cnt_month'].clip(0,20)

X_test = data[data['month_id'] == 34].drop(['cnt_month'], axis=1)

In [6]:
rf_features = [c for c in data.columns if c not in 'cnt_month']

# 데이터 셋 지정
rf_train = X_train[rf_features]
rf_valid = X_valid[rf_features]
rf_test = X_test[rf_features]
 
# 모델 훈련
start = time.time()
rf_model = RandomForestRegressor(n_estimators=50, max_depth=9,random_state=42, n_jobs=-1)
rf_model.fit(rf_train, y_train)
end = time.time()
print(f'소요시간(s) : {end-start}')

소요시간(s) : 2512.6516060829163


In [7]:
# 예측 수행
rf_train_pred = rf_model.predict(rf_train).clip(0,20)
rf_valid_pred = rf_model.predict(rf_valid).clip(0,20)
rf_test_pred = rf_model.predict(rf_test).clip(0, 20) # 실제 타겟값이 클립되어있으므로 동일하게 클립

In [8]:
# 제출 파일 생성
df_test = pd.read_csv('./test.csv')
submission = pd.DataFrame({"ID": df_test.index, "item_cnt_month": rf_test_pred})
submission.to_csv('RF_submission_2.csv', index=False)

In [9]:
# 피쳐 중요도를 가지는 리스트
importances = list(rf_model.feature_importances_)

# (변수, 중요도) 튜플로 구성된 리스트
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(rf_features, importances)]

# 가장 높은 중요도를 가지는 피쳐를 기준으로 내림차순 정렬
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# 변수와 중요도 출력
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: cnt_month_1          Importance: 0.72
Variable: cat_id               Importance: 0.05
Variable: item_id_mean_cnt_1   Importance: 0.05
Variable: shop_id/cat_id_mean_cnt_1 Importance: 0.04
Variable: sales_month          Importance: 0.03
Variable: cnt_month_2          Importance: 0.03
Variable: sub_cat              Importance: 0.02
Variable: month_id             Importance: 0.01
Variable: shop_id              Importance: 0.01
Variable: item_id              Importance: 0.01
Variable: cnt_month_3          Importance: 0.01
Variable: sub_cat_mean_cnt_1   Importance: 0.01
Variable: item_id/city_id_mean_cnt_1 Importance: 0.01
Variable: shop_id/cat_id_mean_cnt_2 Importance: 0.01
Variable: city_id              Importance: 0.0
Variable: main_cat             Importance: 0.0
Variable: year                 Importance: 0.0
Variable: item_id_mean_cnt_2   Importance: 0.0
Variable: item_id_mean_cnt_3   Importance: 0.0
Variable: sub_cat_mean_cnt_2   Importance: 0.0
Variable: sub_cat_mean_cnt_3  

In [10]:
print('Train rmse: ',np.sqrt(mean_squared_error(y_train.clip(0,20), rf_train_pred)))
print('Validation rmse: ',np.sqrt(mean_squared_error(y_valid.clip(0,20), rf_valid_pred)))

Train rmse:  0.8867410979863037
Validation rmse:  0.8299003117495053


In [11]:
print('Test rmse: ',0.95062)

Test rmse:  0.95062
